# Gibbs Sampling for Factorization Machine

This notebook series demonstrate how to implement the Alternating Least Squares (ALS) and Gibbs Sampler for the Factorization Machine (FM) using Python and Julia -- with a view to various applications, in particular to Thompson sampling (TS).

FMs are a type of supervised learning model that can be used for a range of prediction tasks, such as regression, classification, and ranking. They map real-valued features into a low-dimensional latent factor space. This makes them a versatile option for a variety of tasks. ALS is an optimization algorithm for FM, which is used to minimize the loss function of FM. The implementation is based on the paper by Rendle et al. [1, 2].

- Written by Keisuke Morita (GSIS Tohoku Univ.)
- References:
  1. S. Rendle, Z. Gantner, C. Freudenthaler, and L. Schmidt-Thieme, Fast Context-Aware Recommendations with Factorization Machines, in Proceedings of the 34th International ACM SIGIR Conference on Research and Development in Information Retrieval (Association for Computing Machinery, New York, NY, USA, 2011), pp. 635–644.
  2. S. Rendle, Factorization Machines with libFM, ACM Trans. Intell. Syst. Technol. 3, 57:1 (2012).

# 01. Factorization Machine as a Linear Model

In [1]:
import numpy as np
from typing import Optional

## The Factorization Machine

A factorization machine (FM) is given by

$$
\begin{aligned}

f(x; b, w, v)
&= b + \sum_{i=1}^N w_i x_i + \sum_{i \lt j} \sum_{k=1}^K v_{ik} v_{jk} x_i x_j \\

&= b + \sum_{i=1}^N w_i x_i + \frac{1}{2} \sum_{k=1}^K \left( \sum_{i=1}^N \sum_{j=1}^N v_{ik} v_{jk} x_i x_j - \sum_{i=1}^N v_{ik}^2 x_i^2 \right).

\end{aligned}
$$

In [2]:
class FactorizationMachines:
    def __init__(self,
        num_features: int,
        num_factors:  int,
        sigma_b_init: float=0.,
        sigma_w_init: float=1.,
        sigma_v_init: float=1.,
        seed: Optional[int]=None
    ) -> None:
        self.rng = np.random.default_rng(seed)
        b = self.rng.normal(0, sigma_b_init)
        w = self.rng.normal(0, sigma_w_init, num_features)
        v = self.rng.normal(0, sigma_v_init, (num_features, num_factors))
        self.params = {'b': b, 'w': w, 'v': v}

    def predict(self, x: np.ndarray) -> float:
        if x.ndim == 1:
            x = x.reshape(1, -1) # x: (d, n)
        b = self.params['b']     # b: (1)
        w = self.params['w']     # w: (d)
        v = self.params['v']     # v: (d, k)

        bias   = b
            # (1)
        linear = x[:, :] @ w[:]
            # (D, N) @ (N) = (D)
        inter  = 0.5 * np.sum((x[:, :] @ v[:, :]) ** 2 - (x[:, :] ** 2) @ (v[:, :] ** 2), axis=1)
            # (D, K) -> (D)

        result = bias + linear + inter
            # (D)

        if result.shape[0] == 1:
            return float(result[0])
        return result

In [3]:
# test

N = 100
K = 10
D = 5

fm = FactorizationMachines(N, K)

# for x: (N,)
x = np.random.randn(N)
y = fm.predict(x)
print(y)       # float

# for x: (D, N)
x = np.random.choice((0, 1), size=(D, N))
y = fm.predict(x)
print(y.shape) # (5,)


-13.85583253034785
(5,)


## Rewriting an FM as a linear function

- **Lemma** [Rendle+ (2011)]

    A factorization machine is a linear function with respect to every single model parameter $\theta \in \Theta = \{b, w_i, v_{ik}\}$.

    $$
    \begin{aligned}
    f(x) = \theta h_\theta(x) + g_\theta(x)
    \end{aligned}
    $$


Differentiating $f$ by $\theta \in \Theta$ yields

$$
\begin{aligned}
\frac{\partial f}{\partial \theta}
&=
\left\{\begin{aligned}
& 1,
&& \theta=b \\
& x_i,
&& \theta=w_i\\
& x_i \left( \sum_{j=1}^N v_{jk} x_j - v_{ik} x_i \right).
&& \theta = v_{ik}
\end{aligned}\right.
\end{aligned}
$$

By defining $h_\theta(x) \coloneqq \frac{\partial f}{\partial \theta}$ and $g_\theta(x) \coloneqq f(x) - \theta h_\theta(x)$ we obtain

$$
\begin{aligned}
f(x) = \theta h_\theta(x) + g_\theta(x).
\end{aligned}
$$

Remark that the value of $h_\theta(x)$ and $g_\theta(x)$ is independent of the value of $\theta$, that is,

$$
\begin{aligned}
\frac{\partial g_\theta(x)}{\partial \theta}
=
\frac{\partial h_\theta(x)}{\partial \theta}
=
0.
\end{aligned}
$$

In [4]:
def calc_h_b(x: np.ndarray) -> np.ndarray:
    return np.ones(x.shape[0])

def calc_h_w(x: np.ndarray, i: int) -> np.ndarray:
    return x[:, i]

def calc_h_v(x: np.ndarray, v: np.ndarray, i: int, k: int) -> np.ndarray:
    return x[:, i] * (x[:, :] @ v[:, k] - x[:, i] * v[i, k])

def calc_g(f: np.ndarray, h: np.ndarray, p: float) -> np.ndarray:
    return f - h * p

In [5]:
# Confirm that the values of g and h are independent of parameter p.

# for b
h_1 = calc_h_b(x)
g_1 = calc_g(fm.predict(x), h_1, fm.params['b'])

fm.params['b'] = fm.params['b'] - 100

h_2 = calc_h_b(x)
g_2 = calc_g(fm.predict(x), h_2, fm.params['b'])

fm.params['b'] = fm.params['b'] + 100

assert np.isclose(h_1, h_2).all()
assert np.isclose(g_1, g_2).all()

# for w
for i in range(N):
    h_1 = calc_h_w(x, i)
    g_1 = calc_g(fm.predict(x), h_1, fm.params['w'][i])

    fm.params['w'][i] = fm.params['w'][i] - 100

    h_2 = calc_h_w(x, i)
    g_2 = calc_g(fm.predict(x), h_2, fm.params['w'][i])

    fm.params['w'][i] = fm.params['w'][i] + 100

    assert np.isclose(h_1, h_2).all()
    assert np.isclose(g_1, g_2).all()


# for v
from itertools import product
for i, k in product(range(N), range(K)):
    h_1 = calc_h_v(x, fm.params['v'], i, k)
    f_1 = fm.predict(x)
    g_1 = calc_g(f_1, h_1, fm.params['v'][i, k])

    fm.params['v'][i, k] = fm.params['v'][i, k] - 100

    h_2 = calc_h_v(x, fm.params['v'], i, k)
    f_2 = fm.predict(x)
    g_2 = calc_g(f_2, h_2, fm.params['v'][i, k])

    fm.params['v'][i, k] = fm.params['v'][i, k] + 100

    assert np.isclose(h_1, h_2).all()
    assert np.isclose(g_1, g_2).all()

When using FM as a regression model, we assume that the target variable $y$ is given by

$$
\begin{aligned}
y^{(d)}
&= f(x^{(d)}; b, w, v) + \varepsilon^{(d)} \\
&= \theta h_\theta(x^{(d)}) + g_\theta(x^{(d)}) + \varepsilon^{(d)},
\end{aligned}
$$

where

- $y^{(d)}$ is the target variable,
- $x^{(d)}$ is the feature vector,
- $b, w, v$ are the model parameters,
- $\varepsilon^{(d)}$ is the error term,
- $d=1,\ldots,D$ is the index of the data.

To make more clear, we can rewrite the FM as

$$
\begin{aligned}
y_\theta^{(d)} = \theta x_\theta^{(d)} + \varepsilon^{(d)},
\end{aligned}
$$

where

$$
\begin{aligned}
x_\theta^{(d)} &\coloneqq \frac{\partial f(x^{(d)})}{\partial \theta}, \\
y_\theta^{(d)} &\coloneqq y^{(d)} - g_\theta(x^{(d)}) = y^{(d)} - (f^{(d)} - \theta x_\theta^{(d)}).
\end{aligned}
$$

These notations are useful for analyzing the FM as a linear model.

## Speeding up the inference

When using sampling method like MCMC for learning, $x_\theta^{(d)}$ and $y_\theta^{(d)}$ have to be evaluated so many times, which incurs a significant computational cost. Here we show how to reduce this complexity.

To make explicit that $f(x^{(d)})$ is a *value*, not a *function*, we denote it by $f^{(d)} \coloneqq f(x^{(d)})$.

The main idea is to calculate the new $f^{(d)}$ based on its previous value. We save the value of $f^{(d)}$, and every time we sample $\theta^{\rm new}$, we update $f^{(d)}$ using $f^{(d)\rm new} = f^{(d)} + \Delta f^{(d)}$, where

$$
\begin{aligned}
\Delta f^{(d)\rm new}

&\coloneqq
f^{(d)\rm new} - f^{(d)} \\

&=
(\theta^{\rm new} - \theta) \frac{\partial f(x^{(d)})}{\partial \theta} \\

&=
(\theta^{\rm new} - \theta) x_\theta^{(d)}.
\end{aligned}
$$

In [6]:
def calc_df(
    x_theta: np.ndarray,
    param_new: float,
    param_old: float,
):
    return (param_new - param_old) * x_theta

Since $x_\theta^{(d)}$ is given by

$$
\begin{aligned}
x_\theta^{(d)}
&=
\left\{\begin{aligned}
& 1,
&& \theta=b \\
& x_i^{(d)},
&& \theta=w_i\\
& x_i^{(d)} \left( \sum_{j=1}^N v_{jk} x_j^{(d)} - v_{ik} x_i^{(d)} \right),
&& \theta = v_{ik}
\end{aligned}\right.
\end{aligned}
$$

the computational cost of $x_\theta^{(d)}$ is $\mathcal O(1)$ for $\theta = b, w_i$ but for $\theta = v_{ik}$.

To reduce the complexity of $x_{v_{ik}}^{(d)}$, we precalculate the value of $q_k^{(d)}$, which is given by

$$
\begin{aligned}
q_k^{(d)} \coloneqq \sum_{j=1}^N v_{jk} x_j^{(d)},
\end{aligned}
$$

and then we can calculate $x_{v_{ik}}^{(d)}$ by

$$
\begin{aligned}
x_{v_{ik}}^{(d)} = x_i^{(d)} (q_k^{(d)} - v_{ik} x_i^{(d)})
\end{aligned}
$$

in constant time.

$q_k^{(d)}$ can be updated using $q_k^{(d) \rm new} = q_k^{(d)} + \Delta q_k^{(d)}$, where

$$
\begin{aligned}
\Delta q_k^{(d)}
&\coloneqq q_k^{(d) \rm new} - q_k^{(d)} \\
&= x_i^{(d)} (v_{ik}^{\rm new} - v_{ik}),
\end{aligned}
$$

whose time complexity is also $\mathcal O(1)$.

In [7]:
def calc_q_init(
    x: np.ndarray,
    v: np.ndarray
) -> np.ndarray:
    # x: (D, N)
    # v: (N, K)
    return x[:, :] @ v[:, :] # (D, K)

def calc_dq(
    i: int,
    x: np.ndarray,
    v_ik_new: float,
    v_ik_old: np.ndarray,
) -> np.ndarray:
    # v_ik_new: float
    # v: (N, K)
    # x: (D, N)
    return (v_ik_new - v_ik_old) * x[:, i] # (D)

def calc_xy_b(
    f: np.ndarray,
    b: float,
    x_data: np.ndarray,
    y_data: np.ndarray,
):
    # x_data: (D, N)
    # y_data: (D)
    x_b = np.ones(x_data.shape[0])
    y_b = y_data - (f - b * x_b)
    return x_b, y_b

def calc_xy_w(
    f: np.ndarray,
    w: np.ndarray,
    x_data: np.ndarray,
    y_data: np.ndarray,
    i: int
):
    # x_data: (D, N)
    # y_data: (D)
    x_w = x_data[:, i]
    y_w = y_data - (f - x_w * w[i])
    return x_w, y_w

def calc_xy_v_naive(
    f: np.ndarray,
    v: np.ndarray,
    x_data: np.ndarray,
    y_data: np.ndarray,
    i: int,
    k: int
):
    # x_data: (D, N)
    # y_data: (D)
    # v: (N, K)
    x_v = x_data[:, i] * (x_data[:, :] @ v[:, k] - x_data[:, i] * v[i, k])
    y_v = y_data - (f - x_v * v[i, k])
    return x_v, y_v

def calc_xy_v(
    f: np.ndarray,
    q: np.ndarray,
    v: np.ndarray,
    x_data: np.ndarray,
    y_data: np.ndarray,
    i: int,
    k: int
):
    # x_data: (D, N)
    # y_data: (D)
    x_v = x_data[:, i] * (q[:, k] - x_data[:, i] * v[i, k])
    y_v = y_data - (f - x_v * v[i, k])
    return x_v, y_v

In [8]:
# check how much faster the fast version is
import time

def naive_updates(iter, fm, x_data, y_data):
    f = fm.predict(x_data)
    v = fm.params['v']
    for _ in range(iter):
        x, y = calc_xy_v_naive(f, v, x_data, y_data, 0, 0)
        v[0, 0] -= 100
    return x

def fast_updates(iter, fm, x_data, y_data):
    v = fm.params['v']
    f = fm.predict(x_data)
    q = calc_q_init(x_data, v)
    for _ in range(iter):
        x, y = calc_xy_v(f, q, v, x_data, y_data, 0, 0)
        v_ik_new = v[0, 0] - 100
        q[:, 0] += calc_dq(0, x_data, v_ik_new, v[0, 0])
        v[0, 0] = v_ik_new
    return x

iter = 1000

seed = 0
rng  = np.random.default_rng(seed)

N = 100
K = 10
D = 10000
x_data = rng.choice((0, 1), size=(D, N))
y_data = rng.normal(size=(D))

fm = FactorizationMachines(N, K, seed=seed)

time_start = time.time()
h_naive    = naive_updates(iter, fm, x_data, y_data)
time_end   = time.time()
time_naive = time_end - time_start

fm = FactorizationMachines(N, K, seed=seed)

time_start = time.time()
h_fast     = fast_updates(iter, fm, x_data, y_data)
time_end   = time.time()
time_fast  = time_end - time_start

print(f'naive: {time_naive:.5f} [s]')
print(f'fast:  {time_fast:.5f} [s]')
print('-'*34)
print(f'Both returned the same value: {np.isclose(h_fast, h_naive).all()}')

naive: 3.56743 [s]
fast:  0.14770 [s]
----------------------------------
Both returned the same value: True


Now we obtain the whole update rule as follows:

$$
\begin{aligned}
q_k^{(d)}
&=
\sum_{j=1}^N v_{jk} x_j^{(d)},
\\

\Delta f^{(d)}
&= (\theta^{\rm new} - \theta) x_\theta^{(d)},
\\

\Delta q_k^{(d)}
&= x_i^{(d)} (v_{ik}^{\rm new} - v_{ik}),
\\

x_\theta^{(d)}
&=
\left\{\begin{aligned}
& 1,
&& \theta=b \\
& x_i^{(d)},
&& \theta=w_i\\
& x_i^{(d)} \left( q_k^{(d)} - v_{ik} x_i^{(d)} \right).
&& \theta = v_{ik}
\end{aligned}\right.
\end{aligned}
$$

The time complexity is $\mathcal O(1)$ for every $\theta$.